Homework 3 - Predicting survival chance for Titanic passengers

In [41]:
student_name =   "Khachatur Dallakyan"
student_id =     "khda2699"
student_background =  "technical" #

Revised the EDA (Homework 1) - one of the plots has been revised

1) The data is sourced from Titanic Kaggle competition -> https://www.kaggle.com/competitions/titanic/data (for simplicity only train.csv was chosen - data has test/train splits for model training; train.csv is renamed to titanic.csv)


2) The dataset originates from the famous sinking of Titanic in 1912. Dataset contains passenger information on Titanic passengers pressumably before it embarked from the port, primarily used for survival prediction analysis and demographic studies of the event.


3) The dataset consists of two main files: train.csv and test.csv. train.csv was chosen as it additionally has the "Survived" column, as it also has more observations. The data is in CSV format and includes header with feature names. No additional metadata files are provided, but kaggle page has description for the features.


4) The training dataset contains 891 observations (passengers) and 12 features including the target feature 'Survived'. 


5) Yes, the dataset contains numerical features:
- Age: Passenger age in years
- Fare: Ticket price
- SibSp: Number of siblings/spouses aboard the Titanic
- Parch: Number of parents/children aboard the Titanic



6) Yes, the dataset contains both nominal and ordinal categorical features:
Nominal:
- Sex: male/female
- Embarked: Port of embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)
- Cabin: Cabin number
Ordinal:
- Pclass: Passenger class (1, 2, 3)


7) The dataset contains one explicit binary feature:
- Survived: 0 (=No/died) or 1 (=Yes/survived)
Also additional binary features could be engineered from existing features; for example:
- IsChild: Created using Age (1 if age < 18, 0 otherwise)

In [ ]:
import pandas as pd
import numpy as np

df_original = pd.read_csv('titanic.csv')
df_original.info()
df_original.head()

In [ ]:
df = df_original.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
df.head()

PassengerId is just a unique indetifer; Name and Ticket column are inconsistent; difficult to process and hard to find pattern in. Cabin columns has around 700 missing values, which is majority of observation. Therefore droping those columns is good first step for analyses.

In [ ]:
df = df.rename(columns={
    'SibSp': 'Siblings_Spouses_Aboard',
    'Parch': 'Parents_Children_Aboard'
})
df.head()

SibSp and Parch both have unintutive column names, renaming them makes easier to comprehend meaning of those features from the first glance.

In [ ]:
df = df.dropna(subset=['Age'])
df.info()


Age column has some observation with missing values. It was chosen to remove those observation with missing age column. After the filtration 714 observations remain. 

In [ ]:
# Question 1: What percentage of passengers were not adults?
non_adult_passengers = df[df['Age'] < 18]
percentage_non_adult = (len(non_adult_passengers) / len(df) * 100)

print(f"Percentage of not adults among passengers: {percentage_non_adult}")


Presumably not everyone was travelling with their families on titanic, but there were many travelling with families as well, so around 15% of young passenger is within range of expectations. This data gives us information about priority groups who should be saved first.

In [ ]:
# Question 2: What is the average fare price paid by the passengers?
#  What are other statistics related to fare price across all passengers?
fare_stats = df['Fare'].describe()

print("Fare Statistics:")
print(f"Average fare price: £{fare_stats['mean']:.2f}")
print(f"Median fare price: £{fare_stats['50%']:.2f}")
print(f"Standard deviation: £{fare_stats['std']:.2f}")
print(f"Minimum fare price: £{fare_stats['min']:.2f}")
print(f"Maximum fare price: £{fare_stats['max']:.2f}")

Nowadays boat tickets for long trips such as titanic might cost higher, but considering more than a century of inflation average ticket prices are reasonable. Standard deviation is also very high; but that also makes sense as ticket prices differed a lot for different classes.  

In [ ]:
# Question 1: How does the survival rate vary by passenger class?
survival_by_class = df.groupby('Pclass').agg({
    'Survived':['mean']})
survival_by_class.columns = ['Survival Rate']
survival_by_class['Survival Rate'] = (survival_by_class['Survival Rate'] * 100).round(2)

print(survival_by_class)

We can see that higher the class, higher the survival rate, as the 1st class has the highest survival rate followed by middle class and then the third class. It's intuitive, as higher classes had higer social status during victorian era and were given higher priority when evacuating. 

In [ ]:
# Question 2: How does the survival rate vary by gender?
gender_analysis = df.groupby('Sex').agg({'Survived': ['mean']})

gender_analysis.columns = ['Survival Rate']
gender_analysis['Survival Rate'] = (gender_analysis['Survival Rate'] * 100).round(2)

print(gender_analysis)

We can see that females have way higher survivability rate. Again it's intuitive, as women were given priority during evacuation for lifeboats. 

In [50]:
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
# Question 1: How does survival rate vary across passenger classes based on the embarked port?
plt.figure(figsize=(10, 6))
sns.barplot(x='Pclass', y='Survived', hue='Embarked', data=df)
plt.title('Survival Rate by Passenger Class')
plt.xlabel('Passenger Class')
plt.ylabel('Survival Rate')
plt.show()

Boxplot visualization has been chosen since y axis feature Survived is a binary feature. Visualization shows that those who embarked at C = Cherbourg had slightly higher chance of survival. However importantly boxplot visualization also shows the 95% confidence intervals, which in our case is sometimes very large, meaning conclusions about varying survivale rates cannot be certain. 

In [ ]:
# Question 2: How does survival rate vary by gender across different classes?
plt.figure(figsize=(12, 7))
sns.violinplot(x='Pclass', y='Survived', hue='Sex', data=df, split=True, inner='quart')
plt.title('Survival Distribution by Class and Gender')
plt.xlabel('Passenger Class')
plt.ylabel('Survived (0=No, 1=Yes)')
plt.show()

Following recommendations given after homework 1 boxplot was replaced by violinplot. As we can see, female have higher survival rate then male passengers across all classes. However difference is more distinct for the 1st and 2nd class, compared to the 3rd class, where females also have higher casualty rate.

In [ ]:
# Question 3: How does fare distribution vary across passenger classes?
plt.figure(figsize=(10, 6))
sns.boxplot(x='Pclass', y='Fare', data=df)
plt.title('Fare Distribution by Passenger Class')
plt.xlabel('Passenger Class')
plt.ylabel('Fare')
plt.show()

For fair price distibution, boxplot is more approriate visualization tool, as Fare price is a numerical feature and we are interested in mean, standard deviation and outliers. Visualization shows that for the 2nd and 3rd class the price range is low and tickets are priced similarly, but for the 1st class average ticket price is considerably higher with some extremely high outliers. 

In [ ]:
# Question 4: How does family size affect survival?
df['Family_Size'] = df['Siblings_Spouses_Aboard'] + df['Parents_Children_Aboard'] + 1 # +1 to include passenger himself/herself
plt.figure(figsize=(10, 6))
sns.barplot(x='Family_Size', y='Survived', data=df)
plt.title('Survival Rate by Family Size')
plt.xlabel('Total Family Members Aboard')
plt.ylabel('Survival Rate')
plt.show()

Again boxplot visualization has been chosen since y axis feature Survived is a binary feature. Family size is defined as Siblings_Spouses_Aboard + Parents_Children_Aboard + 1 to also include passenger. Visualization shows higher survival rates for those who had 1-3 family member aboard (2 <= Family_Size <= 4), compared to those who were alone aboard (Family_Size = 1). Families larger thay 5 had comparitively larger confidence interval, presumably because of low count of observations, so their average survival rate is less accurate prediction. 

In [ ]:
# Pearson correlation between Age and Fare
correlation = df['Age'].corr(df['Fare'])

print(f"Pearson correlation coefficient between Age and Fare: {correlation:.3f}")

If we approximate the coefficient toward ~0.1; we will have very small positive correlation, with small strength of association.
This does make sense logically, because there are free tickets, which presumably are for children (so lower age correlate with lower prices). However correlation is very small becuase for adults price will rather correlate with class of passengers, rather than age. (For example 24 years old 1st class passenger would pay way more for the tickeet compared to 35 years old 3rd class passenger).

ML classification

Binary target class will be "Survived" [0 (=No/died) or 1 (=Yes/survived)]

In [56]:
features = ['Pclass', 'Sex', 'Age',  'Fare']
x = df[features]
y = df['Survived']

I will build a model that predicts Survival status of Titanic passenger from passenger class, sex, age and ticket price. The reason for selection of this specific input variables is because I beleive this can be the most important features to predict survival status. Individuals from higher class, females and children were given hgiher priority for evacuation, additionally fare ticket price might also contribute to the part of the ship passenger was one, therefore his/her survival rate.

In [ ]:


categorical_features = ['Pclass', 'Sex']
numerical_features = ['Age', 'Fare']

x_encoded = pd.get_dummies(x, columns=categorical_features)
print("Data types after encoding:")
print(x_encoded.dtypes)

print("First 5 rows of encoded dataframe:")
print(x_encoded.head())



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.figure(figsize=(8, 6))
sns.countplot(x=y)
plt.title('Distribution of Survival')
plt.xlabel('Survived (0=No, 1=Yes)')
plt.ylabel('Count')
plt.xticks([0, 1], ['Did not survive', 'Survived'])

unique, counts = np.unique(y, return_counts=True)
class_distribution = dict(zip(unique, counts))
for label, count in class_distribution.items():
    plt.text(label, count, f"{count} ({count/len(y)*100:.1f}%)", 
             ha='center', va='bottom')

plt.tight_layout()
plt.show()


Class imbalance occurs when one class significantly outnumbers others in a dataset. Titanic dataset doesn't suffer from class imbalance. Despite distribution not being 50/50, it's still around 60/40, which is reasonable and no class i not significantly outnumbering the other. Usually imbalances can bias machine learning models toward predicting the majority class, potentially leading to high accuracy but poor performance on the minority class.

In [ ]:
import numpy as np
x = np.asarray(x_encoded)
y = np.asarray(y)

print("x shape:", x.shape)
print("y shape:", y.shape)

In [60]:
from sklearn.model_selection import train_test_split

x_train,  x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)

Stratified partitioning ensures the training and test sets maintain the same proportion of survivors/non-
survivors as the original dataset and the proportion is not modified by sampling. This prevents sampling bias and ensures the model learns meaningful patterns that generalize well to unseen data.

In [61]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

MODELS_TO_TEST = {
    
    "DT_3": DecisionTreeClassifier(max_depth=3),
    "DT_5": DecisionTreeClassifier(max_depth=5),

    "RF_10": RandomForestClassifier(n_estimators=10, max_depth=5),
    "RF_100": RandomForestClassifier(n_estimators=100, max_depth=5),
    
    "SVM_lin": Pipeline([
                      ('scaler', MinMaxScaler()),
                      ('estimator', SVC(kernel='linear'))
                    ]),
    "SVM_rbf": Pipeline([
                      ('scaler', MinMaxScaler()),
                      ('estimator', SVC(kernel='rbf'))
                    ]),
    
    "KNN_3": Pipeline([
                      ('scaler', MinMaxScaler()),
                      ('estimator', KNeighborsClassifier(n_neighbors=3))
                    ]),
    "KNN_7": Pipeline([
                      ('scaler', MinMaxScaler()),
                      ('estimator', KNeighborsClassifier(n_neighbors=7))
                    ]),
    
    "GB_50": GradientBoostingClassifier(n_estimators=50, learning_rate=0.1, max_depth=3),
    "GB_100": GradientBoostingClassifier(n_estimators=100, learning_rate=0.05, max_depth=5)
}


In [ ]:
from sklearn.model_selection import cross_validate

SCORING_METRICS = ["accuracy", "precision_macro", "recall_macro", "f1_macro"]
NUMBER_OF_SPLITS = 10

results_evaluation = pd.DataFrame({
    "classifier_name": [],
    "fit_time": [],
    "score_time": [],
    "test_accuracy": [],
    "test_precision_macro": [],
    "test_recall_macro": [],
    "test_f1_macro": [],
})
for name, classifier in MODELS_TO_TEST.items():
        print(f"Currently training the classifier {name}.")
        
        scores_cv = cross_validate(classifier, x_train, y_train, cv=NUMBER_OF_SPLITS, scoring=SCORING_METRICS)
        
        dict_this_result = {
            "classifier_name": [name],
        }
        
        for metric_name, score_per_fold in scores_cv.items():
            dict_this_result[metric_name] = [score_per_fold.mean()]
            
        this_result = pd.DataFrame(dict_this_result)
        
        results_evaluation = pd.concat([results_evaluation, this_result], ignore_index=True)

In [ ]:
results_evaluation

*Question 1: Which model provides the best balance between computational
efficiency and predictive performance?*

Analysis: Gradient Boosting with 50 estimators (GB_50) offers the best balance with the highest F1 score
(0.815) while requiring moderate training time (0.097s). KNN_3 presents an excellent alternative with
comparable performance (0.799 F1) but dramatically faster training (0.005s) - a 19x speed improvement with
only 2% performance loss.

*Question 2: How does the complexity of Random Forest (RF_100 vs RF_10) affect
both prediction accuracy and evaluation time?*

Analysis: Increasing RF estimators from 10 to 100 results in an 8.3x longer training time (0.025s -> 0.205s)
for only a modest 1.7% F1 score improvement (0.792 -> 0.805). This diminishing return suggests RF_10 may
be sufficient for many applications, especially when computational resources are limited.

*Question 3: Which kernel choice (linear vs RBF) for SVM provides better
performance on this classification task, and at what computational cost?*

Analysis: Surprisingly, the linear SVM outperforms the RBF kernel on F1 score (0.766 vs 0.757) despite taking
longer to train. The RBF kernel shows higher precision (0.842 vs 0.780) but poorer recall (0.749 vs 0.762),
suggesting it's more conservative in its predictions for this particular classification task.

Gradient boosting classifier with 50 estimators performs best with the given data by most of the testing metrics.

In [ ]:
best_model = results_evaluation.loc[results_evaluation['classifier_name'] == 'GB_50']
best_model


Gradient Boost model with 50 estimators performs best with given data with the highest test accuracy (82.7%) and F1 score (0.815). Even though it's the model with highest accuracy among the models tested, according to kaggle submissions there potentially can be models with up to 100% accuracy. Moreover model might have specifically higher accuracy with people from specific category (e.g. Female), while lower with the other categories, which is not tested by the current metrics used to choose the best model. At last, Gradient Boost by its nature is a "black box" (especially compared to Decision trees), which might be problematic for deploying in a scenario related to prediction of human survival chance, considering explainability is important in this case.